# Deform a sphere into a cube


Import the required modules.


In [ ]:
import numpy as np
import pyvista as pv

Define some useful helper functions.


In [ ]:
def pyvista_faces_to_numpy(faces):
    """
    PyVista faces to numpy
    ======================

    Converts a PyVista triangles array to a numpy array of shape (n_faces, 3).
    Assumes all faces are triangles (first value in each group is 3).
    """
    n_faces = len(faces) // 4
    return faces.reshape((n_faces, 4))[:, 1:]


def numpy_faces_to_pyvista(faces):
    """
    Numpy faces to PyVista
    ======================

    Converts a numpy triangles array of shape (n_faces, 3) to the format expected by PyVista.
    """
    n_faces = faces.shape[0]
    return np.hstack([np.full((n_faces, 1), 3, dtype=int), faces]).ravel()

## 1. Create a sphere


### 1.1 Make a 2D mesh


We will define the vertices in spherical, $(r, \theta, \phi)$, coordinates. For a sphere, $r=1$, so if we plot the mesh, it will look like a 2D plane. We can perform a tessellation of the mesh using the `delaunay_2d()` function; it should get the tessellation mostly correct, since the surface is 2D.


In [ ]:
# Generate arrays of theta, phi values for all points, excluding the poles.
N = 20
theta = np.linspace(0, np.pi, N)[1:-1]
phi = np.linspace(0, 2 * np.pi, 2 * N + 1)[:-1]

# Make an array of vertices for the bulk of the sphere.
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)
sphere_vertices_spherical_no_poles = np.c_[
    r_v.reshape(-1), theta_v.reshape(-1), phi_v.reshape(-1)
]

# Add the poles to the array of vertices.
sphere_vertices_spherical = np.zeros(
    (sphere_vertices_spherical_no_poles.shape[0] + 2, 3), dtype=float
)
sphere_vertices_spherical[0, :] = [1.0, 0.0, 0.0]
sphere_vertices_spherical[1:-1, :] = sphere_vertices_spherical_no_poles
sphere_vertices_spherical[-1, :] = [1.0, np.pi, 0.0]

# Create a PyVista mesh, and perform and plot a Delaunay tessellation.
sphere_mesh_spherical_delaunay = pv.PolyData(sphere_vertices_spherical).delaunay_2d()
sphere_mesh_spherical_delaunay.plot(show_edges=True)

### 1.2 Modify the faces


The Delaunay tessellation correctly tessellated some features of our mesh, but not others. We want the North pole ($\theta=0$) and the South pole ($\theta=\pi$) to be connnected to all of the adjacent $\theta$ values. This is handled by the `add_pole_faces()` function. We also need to connect the points with the largest $\phi$ values to the points at $\phi=0$. This is handled by the `wrap_phi()` function. Both of these functions are lumped into the `correct_mesh()` function, which should produce a correctly tessellated 2D mesh when called. The 2D mesh looks quite weird when plotted, as $\phi=0$ is equivalent to $\phi=2 \pi$, so there are faces connecting the largest $\phi$ edge to the $\phi = 0$ edge; this disappears when we plot the mesh in Cartesian coordinates.


In [ ]:
def add_pole_faces(mesh, num_azimuthal_angles):
    """
    Add pole faces
    ==============

    Appends all of the faces involving the poles to the mesh.
    This function assumes that `wrap_phi` has not been called.
    """
    # Extract the faces from the PyVista mesh.
    faces = pyvista_faces_to_numpy(mesh.faces)
    max_vertex = np.max(faces)

    # Remove all faces involving the poles.
    bulk_faces = faces[
        ~((faces == 0).any(axis=1) | (faces == np.max(faces)).any(axis=1))
    ]

    # Generate faces made by the North pole.
    north_faces = np.zeros((num_azimuthal_angles - 1, 3), dtype=int)
    for i in range(num_azimuthal_angles - 1):
        north_faces[i, :] = np.array([i + 2, 0, i + 1])

    # Generate faces made by the South pole.
    south_faces = np.zeros((num_azimuthal_angles - 1, 3), dtype=int)
    for i in range(num_azimuthal_angles - 1):
        south_faces[i, :] = np.array(
            [
                max_vertex,
                max_vertex - num_azimuthal_angles + i + 1,
                max_vertex - num_azimuthal_angles + i,
            ]
        )

    return pv.PolyData(
        mesh.points,
        numpy_faces_to_pyvista(np.vstack((north_faces, bulk_faces, south_faces))),
    )


def wrap_phi(mesh, num_polar_angles, num_azimuthal_angles):
    """
    Wrap phi
    ========

    This function connects the points with the largest phi values to the points phi = 0.
    This function assumes that `add_pole_faces` has already been called.
    """
    # Extract the faces from the PyVista mesh.
    faces = pyvista_faces_to_numpy(mesh.faces)
    max_vertex = np.max(faces)

    # Wrap the phi coordinate in the bulk.
    wrapping_faces = np.zeros((2 * (num_polar_angles - 1) + 2, 3), dtype=int)
    for i in range(num_polar_angles - 1):
        # Make the first triangle.
        wrapping_faces[2 * i, :] = np.array(
            [
                (i + 2) * num_azimuthal_angles,
                (i * num_azimuthal_angles) + 1,
                (i + 1) * num_azimuthal_angles,
            ]
        )

        # Make the second triangle.
        wrapping_faces[(2 * i) + 1, :] = np.array(
            [
                (i + 2) * num_azimuthal_angles,
                (i + 1) * num_azimuthal_angles + 1,
                (i * num_azimuthal_angles) + 1,
            ]
        )

    # Wrap the phi coordinates at the poles.
    wrapping_faces[-2, :] = np.array([1, num_azimuthal_angles, 0])
    wrapping_faces[-1, :] = np.array(
        [max_vertex, max_vertex - 1, max_vertex - num_azimuthal_angles]
    )

    return pv.PolyData(
        mesh.points, numpy_faces_to_pyvista(np.vstack((faces, wrapping_faces)))
    )


def correct_mesh(mesh, num_polar_angles, num_azimuthal_angles):
    """
    Correct mesh
    ============

    Corrects the tessellation produced by the `delaunay_2d()` function, and returns a new mesh.
    """
    mesh_1 = add_pole_faces(mesh, num_azimuthal_angles)
    mesh_2 = wrap_phi(mesh_1, num_polar_angles, num_azimuthal_angles)
    return mesh_2

In [ ]:
num_polar_angles = len(theta)
num_azimuthal_angles = len(phi)
sphere_mesh_spherical_custom = correct_mesh(
    sphere_mesh_spherical_delaunay, num_polar_angles, num_azimuthal_angles
)
sphere_mesh_spherical_custom.plot(show_edges=True)

### 1.3 Map the 2D mesh to a 3D surface


Now that the 2D mesh is correctly tessellated, we can map our spherical, $(r, \theta, \phi)$ vertices to Cartesian, $(x, y, z)$, vertices, while keeping the same tessellation. This should result in a correctly tessellated sphere. We can also calculate vertex normals using the Cartesian coordinates of the vertices, and add the vertex normals to our mesh as point data.


In [ ]:
# Make an array storing the vertices in Cartesian coordinates.
x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)
sphere_vertices_cartesian_no_poles = np.c_[
    x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)
]

# Add the poles to the array of vertices.
sphere_vertices_cartesian = np.zeros(
    (sphere_vertices_cartesian_no_poles.shape[0] + 2, 3), dtype=float
)
sphere_vertices_cartesian[0, :] = [0.0, 0.0, 1.0]
sphere_vertices_cartesian[1:-1, :] = sphere_vertices_cartesian_no_poles
sphere_vertices_cartesian[-1, :] = [0.0, 00, -1.0]

# Calculate the vertex normals.
center = np.array([0, 0, 0])
normals = sphere_vertices_cartesian - center[None, :]
normals = normals / np.linalg.norm(normals, axis=1)[:, None]

# Create and plot a new mesh, using the triangles array generated for the 2D mesh.
sphere_mesh_cartesian_custom = pv.PolyData(
    sphere_vertices_cartesian, sphere_mesh_spherical_custom.faces
)
sphere_mesh_cartesian_custom.point_data["normals"] = normals
sphere_mesh_cartesian_custom.plot(show_edges=True, color="lightblue")

### 1.4 Comparison to Delaunay 3D tessellation.


We can directly tessellating the vertices in 3D using the `delaunay_3d()` method. However, the second mesh is not rotationally symmetric about the $z$ axis - this is most obvious when looking down the barrel of the $x$ axis, with the $z$ axis pointing upwards. For the deformation, we will keep the first mesh, as we want to mesh to be as symmetric as possible.


In [ ]:
sphere_mesh_cartesian_delaunay = pv.PolyData(
    sphere_mesh_cartesian_custom.points
).delaunay_3d()
sphere_mesh_cartesian_delaunay.plot(show_edges=True, color="lightblue")

## 2. Create a cube


### 2.1 Make a 2D mesh


We will start by generating 6 square meshes, one for each face of the cube. We will tesselate each face individually using the `delaunay_2d()` method.


In [ ]:
# Generate arrays for x, y values
N = 10
L = 2
x = np.linspace(-L / 2, L / 2, N)
y = np.linspace(-L / 2, L / 2, N)
z = np.array([0])

# Make an array of vertices for a square.
x_v, y_v, z_v = np.meshgrid(x, y, z)
square_vertices_2d = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

In [ ]:
# Generate vertices and triangles for the x = -L/2 face.
vertices_x_min = np.zeros(square_vertices_2d.shape)
vertices_x_min[:, 0] = -L / 2
vertices_x_min[:, 1] = square_vertices_2d[:, 1]
vertices_x_min[:, 2] = square_vertices_2d[:, 0]
faces_x_min = pyvista_faces_to_numpy(pv.PolyData(vertices_x_min).delaunay_2d().faces)

# Generate vertices and triangles for the x = +L/2 face.
vertices_x_max = np.zeros(square_vertices_2d.shape)
vertices_x_max[:, 0] = +L / 2
vertices_x_max[:, 1] = square_vertices_2d[:, 1]
vertices_x_max[:, 2] = square_vertices_2d[:, 0]
faces_x_max = pyvista_faces_to_numpy(pv.PolyData(vertices_x_max).delaunay_2d().faces)

# Generate vertices and triangles for the y = -L/2 face.
vertices_y_min = np.zeros(square_vertices_2d.shape)
vertices_y_min[:, 0] = square_vertices_2d[:, 0]
vertices_y_min[:, 1] = -L / 2
vertices_y_min[:, 2] = square_vertices_2d[:, 1]
faces_y_min = pyvista_faces_to_numpy(pv.PolyData(vertices_y_min).delaunay_2d().faces)

# Generate vertices and triangles for the y = +L/2 face.
vertices_y_max = np.zeros(square_vertices_2d.shape)
vertices_y_max[:, 0] = square_vertices_2d[:, 0]
vertices_y_max[:, 1] = L / 2
vertices_y_max[:, 2] = square_vertices_2d[:, 1]
faces_y_max = pyvista_faces_to_numpy(pv.PolyData(vertices_y_max).delaunay_2d().faces)

# Generate vertices and triangles for the z = -L/2 face.
vertices_z_min = np.zeros(square_vertices_2d.shape)
vertices_z_min[:, 0] = square_vertices_2d[:, 1]
vertices_z_min[:, 1] = square_vertices_2d[:, 0]
vertices_z_min[:, 2] = -L / 2
faces_z_min = pyvista_faces_to_numpy(pv.PolyData(vertices_z_min).delaunay_2d().faces)

# Generate vertices and triangles for the z = +L/2 face.
vertices_z_max = np.zeros(square_vertices_2d.shape)
vertices_z_max[:, 0] = square_vertices_2d[:, 1]
vertices_z_max[:, 1] = square_vertices_2d[:, 0]
vertices_z_max[:, 2] = L / 2
faces_z_max = pyvista_faces_to_numpy(pv.PolyData(vertices_z_max).delaunay_2d().faces)

### 2.2 Combine the 2D meshes to make a cube


We will combine the 6 square meshes to make a cube. Once we have created a 3D mesh, we will use the `pyvista.PolyData.clean()` method to remove duplicate vertices, and update the triangles array. We can also calculate vertex normals, and add them to our mesh as point data.


In [ ]:
# Create an array of vertices.
cube_vertices_cartesian = np.vstack(
    (
        vertices_x_min,
        vertices_x_max,
        vertices_y_min,
        vertices_y_max,
        vertices_z_min,
        vertices_z_max,
    )
)

# Create an array of faces.
num_vertices = square_vertices_2d.shape[0]
cube_faces = numpy_faces_to_pyvista(
    np.vstack(
        (
            faces_x_min,
            faces_x_max + num_vertices,
            faces_y_min + 2 * num_vertices,
            faces_y_max + 3 * num_vertices,
            faces_z_min + 4 * num_vertices,
            faces_z_max + 5 * num_vertices,
        )
    )
)

# Create a 3D mesh.
cube_mesh_cartesian_custom = pv.PolyData(cube_vertices_cartesian, cube_faces)
cube_mesh_cartesian_custom.clean()

# Calculate the vertex normals, and add them to the mesh as point data.
center = np.array([0, 0, 0])
normals = cube_mesh_cartesian_custom.points - center[None, :]
normals = normals / np.linalg.norm(normals, axis=1)[:, None]
cube_mesh_cartesian_custom.point_data["normals"] = normals

# Plot the mesh.
cube_mesh_cartesian_custom.plot(show_edges=True, color="lightblue")

### 2.3 Comparison with Delaunay 3D tessellation.


We can directly tessellate the cube using the `delaunay_3d()` method; however, the tessellation produced by this method is not as good as by individually tessellating each face with the `delaunay_2d()` method.


In [ ]:
cube_mesh_cartesian_delaunay = pv.PolyData(
    cube_mesh_cartesian_custom.points
).delaunay_3d()
cube_mesh_cartesian_delaunay.plot(show_edges=True, color="lightblue")

## 3. Deform a sphere into a cube
